In [1]:
# ## Tallahassee AQI Data Retrieval

# ### 1. Setup

# Import required modules and constants
import EPA_Requests_2 as EPA
import pandas as pd

# Tallahassee coordinates
TALLAHASSEE_COORDS = (30.4383, -84.2807)

# AQI time range (example: January 1, 2023 to December 31, 2023)
START_DATE = "20230101"  # YYYYMMDD format
END_DATE = "20231231"

# AQI parameters for gaseous and particulate pollutants
AQI_PARAMS = EPA.AQI_PARAMS_GASEOUS + EPA.AQI_PARAMS_PARTICULATES


In [6]:
AQS_REQUEST_TEMPLATE = {
    "email":      "kateroll@gmail.com",     
    "key":        "silverfox66",      
    "state":      "12",     # the two digit state FIPS # as a string
    "county":     "073",     # the three digit county FIPS # as a string
    "begin_date": "",     # the start of a time window in YYYYMMDD format
    "end_date":   "",     # the end of a time window in YYYYMMDD format, begin_date and end_date must be in the same year
    "minlat":    29.71366231884058,
    "maxlat":    31.162937681159423,
    "minlon":    -85.19645091575092,
    "maxlon":    -83.36494908424908,
    "param":     "",     # a list of comma separated 5 digit codes, max 5 codes requested
    "pclass":    ""      # parameter class is only used by the List calls
}

In [2]:
# ### 2. Calculate Bounding Box for Tallahassee

# Generate a 50-mile bounding box around Tallahassee
bounding_box = EPA.calculate_bounding_box(TALLAHASSEE_COORDS[0], TALLAHASSEE_COORDS[1], scale=2.0)  # 50 miles
print("Bounding Box Coordinates:", bounding_box)


Bounding Box Coordinates: {'min_lat': 29.71366231884058, 'max_lat': 31.162937681159423, 'min_lon': -85.19645091575092, 'max_lon': -83.36494908424908}


In [4]:
# ### 3. Get Monitoring Stations within the Bounding Box

# Fetch the monitoring stations within the bounding box
stations_df = EPA.request_monitors_B(bounding_box=bounding_box, param=','.join(AQI_PARAMS),
                                   begin_date=START_DATE, end_date=END_DATE)
print("Monitoring Stations within 50 miles of Tallahassee:")
stations_df.head()


KeyError: 'minlat'

In [7]:
# ### 3. Get Monitoring Stations within the Bounding Box

# Define a function to request monitors using the AQS_REQUEST_TEMPLATE
def request_monitors_with_template(template: dict) -> pd.DataFrame:
    """
    Request monitoring stations based on the template, updating the API action.

    Parameters
    ----------
    template : dict
        Template containing all required parameters for the API request.

    Returns
    -------
    pd.DataFrame
        DataFrame with monitoring station information if successful.
    """
    request_data = template.copy()
    endpoint_action = EPA.API_ACTION_MONITORS_BOX.format(**request_data)
    request_url = EPA.API_REQUEST_URL + endpoint_action
    response = EPA.make_api_request(request_url)

    if response and "Data" in response:
        return pd.DataFrame(response["Data"])
    return pd.DataFrame()  # Empty DataFrame if no data

# Fetch monitoring stations
stations_df = request_monitors_with_template(AQS_REQUEST_TEMPLATE)
print("Monitoring Stations within 50 miles of Tallahassee:")
stations_df.head()

Error fetching data: 400 Client Error: Bad Request for url: https://aqs.epa.gov/data/api/monitors/byBox?email=kateroll@gmail.com&key=silverfox66&param=&minlat=29.71366231884058&maxlat=31.162937681159423&minlon=-85.19645091575092&maxlon=-83.36494908424908&bdate=&edate=
Monitoring Stations within 50 miles of Tallahassee:


""


In [5]:
bounding_box_params = {
    "minlat": bounding_box["min_lat"],
    "maxlat": bounding_box["max_lat"],
    "minlon": bounding_box["min_lon"],
    "maxlon": bounding_box["max_lon"]
}

print("Bounding Box Coordinates:", bounding_box_params)

Bounding Box Coordinates: {'minlat': 29.71366231884058, 'maxlat': 31.162937681159423, 'minlon': -85.19645091575092, 'maxlon': -83.36494908424908}


In [5]:
stations_df = EPA.request_monitors(bounding_box=bounding_box_params, param=','.join(AQI_PARAMS),
                                   begin_date=START_DATE, end_date=END_DATE)
print("Monitoring Stations within 50 miles of Tallahassee:")
stations_df.head()

Error fetching data: 400 Client Error: Bad Request for url: https://aqs.epa.gov/data/api/monitors/byBox?email=your_default_email&key=your_default_api_key&param=O3,NO2,SO2,PM2.5,PM10&minlat=29.71366231884058&maxlat=31.162937681159423&minlon=-85.19645091575092&maxlon=-83.36494908424908&bdate=20230101&edate=20231231
Monitoring Stations within 50 miles of Tallahassee:


""


In [6]:
# Retry with a single AQI parameter to test the API
stations_df = EPA.request_monitors(bounding_box=bounding_box_params, param="O3",
                                   begin_date=START_DATE, end_date=END_DATE)
print("Monitoring Stations within 50 miles of Tallahassee:")
stations_df.head()

Error fetching data: 400 Client Error: Bad Request for url: https://aqs.epa.gov/data/api/monitors/byBox?email=your_default_email&key=your_default_api_key&param=O3&minlat=29.71366231884058&maxlat=31.162937681159423&minlon=-85.19645091575092&maxlon=-83.36494908424908&bdate=20230101&edate=20231231
Monitoring Stations within 50 miles of Tallahassee:


""
